In [44]:
import yfinance as yf
import pandas as pd

TICKER = "AAPL"
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

# Download data
data = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=False)
print("Raw columns:", data.columns)

# Flatten MultiIndex columns if needed
if isinstance(data.columns, pd.MultiIndex):
    # Join tuple elements with underscore
    data.columns = [
        '_'.join([str(el) for el in col if el]) for col in data.columns
    ]

# Remove ticker or extra prefixes if present (sometimes shows as "AAPL_Open", etc.)
for prefix in [f"{TICKER}_", "AAPL_", "AAPL"]:
    data.columns = [col.replace(prefix, "") for col in data.columns]

# Now capitalize columns for Backtesting.py compatibility
data.columns = [col.title() for col in data.columns]
print("Cleaned columns:", data.columns)

# Use 'Adj Close' for 'Close' if needed
if "Adj Close" in data.columns and "Close" not in data.columns:
    data["Close"] = data["Adj Close"]

# Only keep necessary columns and drop missing data
required_cols = ["Open", "High", "Low", "Close", "Volume"]
present_cols = [col for col in required_cols if col in data.columns]
data = data[present_cols].dropna()

print("Final columns for backtesting:", data.columns)
print(data.head())


[*********************100%***********************]  1 of 1 completed

Raw columns: MultiIndex([('Adj Close', 'AAPL'),
            (    'Close', 'AAPL'),
            (     'High', 'AAPL'),
            (      'Low', 'AAPL'),
            (     'Open', 'AAPL'),
            (   'Volume', 'AAPL')],
           names=['Price', 'Ticker'])
Cleaned columns: Index(['Adj Close_', 'Close_', 'High_', 'Low_', 'Open_', 'Volume_'], dtype='object')
Final columns for backtesting: Index([], dtype='object')
Empty DataFrame
Columns: []
Index: [2020-01-02 00:00:00, 2020-01-03 00:00:00, 2020-01-06 00:00:00, 2020-01-07 00:00:00, 2020-01-08 00:00:00]


In [45]:
from backtesting import Backtest, Strategy
from backtesting.test import SMA

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and self.sma1[-1] > self.sma2[-1]):
            self.buy()
        elif (self.sma1[-2] > self.sma2[-2] and self.sma1[-1] < self.sma2[-1]):
            self.sell()


In [46]:
print(data.columns)
print(data.head())
print("Rows:", len(data))


Index([], dtype='object')
Empty DataFrame
Columns: []
Index: [2020-01-02 00:00:00, 2020-01-03 00:00:00, 2020-01-06 00:00:00, 2020-01-07 00:00:00, 2020-01-08 00:00:00]
Rows: 1257


In [47]:
# Lowercase and remove spaces, then capitalize first letter
def clean_col(col):
    col = str(col)
    col = col.split('_')[-1]   # Remove prefix if present
    col = col.lower().capitalize()
    return col

data.columns = [clean_col(col) for col in data.columns]
print("Renamed columns:", data.columns)


Renamed columns: Index([], dtype='object')


In [48]:
if "Adjclose" in [c.lower().replace(' ', '') for c in data.columns]:
    # Use 'Adj Close' if standard 'Close' is missing
    data["Close"] = data[[c for c in data.columns if "adjclose" in c.lower().replace(' ', '')][0]]

# If Volume is missing, you can skip it, but Backtesting.py runs best with it.
final_cols = ["Open", "High", "Low", "Close", "Volume"]
available = [col for col in final_cols if col in data.columns]
data = data[available].dropna()
print("Cleaned columns:", data.columns)


Cleaned columns: Index([], dtype='object')


In [49]:
if set(["Open", "High", "Low", "Close"]).issubset(data.columns):
    print("Ready for Backtesting")
else:
    print("Still missing required columns!", data.columns)


Still missing required columns! Index([], dtype='object')


In [55]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.test import SMA

TICKER = "AAPL"
START_DATE = "2020-01-01"
END_DATE = "2024-12-31"

# Step 1: Download
data = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=False)

# Step 2: Flatten columns if MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = ['_'.join(map(str, col)).replace(" ", "") for col in data.columns]

print("Flattened columns:", data.columns)
print(data.head())

# Step 3: Explicitly rename columns
colmap = {
    f"Open_{TICKER}": "Open",
    f"High_{TICKER}": "High",
    f"Low_{TICKER}": "Low",
    f"Close_{TICKER}": "Close",
    f"Volume_{TICKER}": "Volume",
    f"Adj Close_{TICKER}": "Close"  # Use 'Adj Close' for 'Close' if needed
}
# Only keep keys that exist
actual_map = {k: v for k, v in colmap.items() if k in data.columns}
data = data.rename(columns=actual_map)

required_cols = ["Open", "High", "Low", "Close", "Volume"]
present_cols = [col for col in required_cols if col in data.columns]
data = data[present_cols].dropna()

print("Final columns for backtesting:", data.columns)
print(data.head())

# Step 4: Define strategy
class SmaCross(Strategy):
    n1 = 10
    n2 = 20
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    def next(self):
        if self.sma1[-2] < self.sma2[-2] and self.sma1[-1] > self.sma2[-1]:
            self.buy()
        elif self.sma1[-2] > self.sma2[-2] and self.sma1[-1] < self.sma2[-1]:
            self.sell()

# Step 5: Backtest and optimize
bt = Backtest(
    data,
    SmaCross,
    cash=10_000,
    commission=0.002,
    exclusive_orders=True
)

result = bt.optimize(
    n1=range(5, 31, 5),
    n2=range(20, 101, 10),
    constraint=lambda p: p.n2 > p.n1,
    maximize='Sharpe Ratio'
)


# Step 6: Output results
print("\n--- Optimized Strategy Results ---")
print(f"Best n1: {result._strategy.n1}")
print(f"Best n2: {result._strategy.n2}")
print(f"Sharpe Ratio: {result['Sharpe Ratio']:.3f}")
print(f"Profit Factor: {result['Profit Factor']:.3f}")
print(f"Return [%]: {result['Return [%]']:.2f}%")
print(f"Max. Drawdown [%]: {result['Max. Drawdown [%]']:.2f}%")
print(f"Total Trades: {result['# Trades']}")



[*********************100%***********************]  1 of 1 completed

Flattened columns: Index(['AdjClose_AAPL', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL',
       'Volume_AAPL'],
      dtype='object')
            AdjClose_AAPL  Close_AAPL  High_AAPL   Low_AAPL  Open_AAPL  \
Date                                                                     
2020-01-02      72.468269   75.087502  75.150002  73.797501  74.059998   
2020-01-03      71.763710   74.357498  75.144997  74.125000  74.287498   
2020-01-06      72.335571   74.949997  74.989998  73.187500  73.447502   
2020-01-07      71.995377   74.597504  75.224998  74.370003  74.959999   
2020-01-08      73.153511   75.797501  76.110001  74.290001  74.290001   

            Volume_AAPL  
Date                     
2020-01-02    135480400  
2020-01-03    146322800  
2020-01-06    118387200  
2020-01-07    108872000  
2020-01-08    132079200  
Final columns for backtesting: Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
                 Open       High        Low      Close     Vol

Backtest.optimize:   0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/backtesting/backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
/usr/local/lib/python3.12/dist-packages/backtesting/backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)


Backtest.run:   0%|          | 0/1227 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/1157 [00:00<?, ?bar/s]

/usr/local/lib/python3.12/dist-packages/backtesting/backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)


Backtest.run:   0%|          | 0/1237 [00:00<?, ?bar/s]


--- Optimized Strategy Results ---
Best n1: 5
Best n2: 20
Sharpe Ratio: 0.521
Profit Factor: 1.715
Return [%]: 133.39%
Max. Drawdown [%]: -28.54%
Total Trades: 59


/usr/local/lib/python3.12/dist-packages/backtesting/backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


In [56]:
print(result.iloc[0])


2020-01-02 00:00:00


In [57]:
bt.plot()

GridPlot(id='p1349', ...)